## 機械学習

## データ教師なし学習　クラスタリング

In [ ]:
#事前準備　クラスタリングする際に必要ない情報は削除しておく 今回は事前にいらない情報を削除したデータを　data_clustering　とする

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
data_clustering_sc = sc.fit_transform(data_clustering)

kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit(data_clustering_sc)
data_clustering["cluster"] = clusters.labels_
print(data_clustering["cluster"].unique())
data_clustering.head()

In [ ]:
## クラスタリングの内容の確認

In [ ]:
data_clustering.groupby("cluster").count()

In [ ]:
## クラスタリングしたデータを可視化

In [ ]:
from sklearn.decomposition import PCA
X = data_clustering_sc
pca = PCA(n_components=2)
pca.fit(X)
x_pca = pca.transform(X)
pca_df = pd.DataFrame(x_pca)
pca_df["cluster"] = data_clustering["cluster"]

import matplotlib.pyplot as plt
%matplotlib inline
for i in data_clustering["cluster"].unique():
    tmp = pca_df.loc[pca_df["cluster"]==i]
    plt.scatter(tmp[0], tmp[1])

## 教師あり学習 ライナーモデル

In [ ]:
from sklearn import linear_model
import sklearn.model_selection
model = linear_model.LinearRegression()

X = data[["独立変数","独立変数","独立変数","独立変数","独立変数"]]
y = data["従属変数"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)
model.fit(X_train, y_train)

In [ ]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

In [ ]:
## 中身の確認　寄与率や偏回帰係数などを可視化→ブラックボックス化の解消

In [ ]:
coef = pd.DataFrame({"feature_names":X.columns, "coefficient":model.coef_})
coef

In [ ]:
##　予測する　ノック40

In [ ]:
x1 = [3, 4, 4, 6, 8, 7, 8]
x2 = [2, 2, 3, 3, 4, 6, 8]
x_pred = [x1, x2]
model.predict(x_pred)

## 教師あり学習　決定木　

In [ ]:
#事前準備として変数を0 or 1　などの数字で表せるようにしなければならない

In [ ]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()
#元々あったpredict_dataを0 or 1 数字だけで構築されたpredict_dataに変換した

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

Yes = predict_data.loc[predict_data["分類分けに使う列名"]==1]
No= predict_data.loc[predict_data["分類分けに使う列名"]==0].sample(len(Yes))
#sample(len(Yes))はデータの量をそろえるためにやる　Noの方が少ない場合は Yesの末尾にsample(len(No))を咥える

X = pd.concat([Yes, No], ignore_index=True)
y = X["分類分けに使う列名"]
del X["分類分けに使う列名"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

In [ ]:
# モデルの実行

In [ ]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

In [ ]:
#モデルの評価

In [ ]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

In [ ]:
##　チューニング方法

In [ ]:
X = pd.concat([Yes, No], ignore_index=True)
y = X["分類分けに使う列名"]
del X["分類分けに使う列名"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
#枝分かれの回数を5回にした
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))